In [2]:
%matplotlib inline
import pandas as pd

In [3]:
compras_df = pd.read_csv('data/clean_compras.csv')

In [4]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una expansion de datos que tenemos, fecha por ejemplo

In [5]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [6]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day)

In [7]:
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month)

In [8]:
compras_df['ANO'] = compras_df['FECHA'].apply(lambda x: x.year)

## Proyectemos unas variables contra otras

In [9]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [32]:
beneficiario_sales

{'CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION': 266,
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.': 101,
 'CAR ONE MONTERREY, S.A. DE C.V.': 78,
 'PEREZ RIOS RAMIRO': 58,
 'S.I.M.E.P.R.O.D.E.': 50,
 'INTERASFALTOS, S.A. DE C.V.': 45,
 'GRUPO COYOTE MONTERREY, S.A. DE C.V.': 43,
 'SERVICIO TECNICO CHURUBUSCO, S.A. DE C.V.': 30,
 'CFE SUMINISTRADOR DE SERVICIOS BASICOS': 29,
 'CORDERO MACIAS SILVESTRE GUADALUPE': 27,
 'SIERRA MADRE RACING, S.A. DE C.V.': 21,
 'CLIMEEZ DEL NORTE,S.A.DE C.V.': 21,
 'GS GAS Q, S.A. DE C.V.': 18,
 'INMOBILIARIA ESTRUCTURAL, S.A. DE C.V.': 18,
 'SECRETARIA DE FINANZAS Y TESORERIA GRAL DEL EDO DE NL': 18,
 'GARCIA ZAVALA EDGAR ALEJANDRO JOBERAGAN': 17,
 'MUNICIPIO DE LA CIUDAD DE MONTERREY': 16,
 'SI VALE MEXICO, S.A. DE C.V.': 16,
 'MOTOR DEAL, S.A. DE C.V.': 15,
 'VALUE ARRENDADORA,S.A.DE C.V. SOFOM ER VALUE GPO.FIN.': 14,
 'LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.': 13,
 'MILENIO DIARIO, S.A. DE C.V.': 12,
 'REGIO ASFALTOS Y CONCRETOS, SA. DE CV.': 11,
 

In [10]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [11]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x]) # va a agregar la cantidad que se repite el beneficiacio en la columna

In [34]:
compras_df['BENEFICIARIO']

0       LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.
1       LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.
2       LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.
3       LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.
4       LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.
                             ...                      
1535                              SEGOVIA PAEZ MARCELO
1536                              SEGOVIA PAEZ MARCELO
1537                      TORRES ACOSTA ROBERTO CARLOS
1538                      TOVAR CEDILLO JAVIER ALFONSO
1539                  VILLARREAL CARDENAS NANCY ESTHER
Name: BENEFICIARIO, Length: 1540, dtype: object

In [41]:
beneficiario_sales.get('LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.')

13

In [33]:
compras_df['BENEFICIARIO_SALES']

0       13
1       13
2       13
3       13
4       13
        ..
1535     2
1536     2
1537     2
1538     1
1539     2
Name: BENEFICIARIO_SALES, Length: 1540, dtype: int64

In [12]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()

In [13]:
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [14]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [15]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x]) 

## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [16]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('data/unique_beneficiarios.csv', index=None)

In [17]:
#Cargar
linkedin_mty = pd.read_csv('data/linkedin_mty.csv')

In [18]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [19]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [20]:
compras_df['HAS_LINKEDIN_SITE'] 

0       True
1       True
2       True
3       True
4       True
        ... 
1535    True
1536    True
1537    True
1538    True
1539    True
Name: HAS_LINKEDIN_SITE, Length: 1540, dtype: bool

In [21]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

HAS_LINKEDIN_SITE
True     1392
False     148
Name: count, dtype: int64

In [22]:
compras_df.to_csv('data/compras_df_enriched.csv', index=None)